<a href="https://colab.research.google.com/github/MinkyuRamen/KubigFinancialProject/blob/main/ky/FINRL_Application_KY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prediction 없이

### Package Install

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

c:\Users\se99a\anaconda3\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


### Create Folders

In [2]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])



In [3]:
# from config.py, TRAIN_START_DATE is a string
TRAIN_START_DATE
# from config.py, TRAIN_END_DATE is a string
TRAIN_END_DATE

'2020-07-31'

In [4]:
TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2021-10-01'
TRADE_START_DATE = '2021-10-01'
TRADE_END_DATE = '2023-05-05'

In [5]:
tickers = ['XLB','XLE','XLF','XLI','XLK','XLP','XLU','XLV','XLY']

In [6]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = tickers).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (30213, 8)


In [7]:
df

,date,open,high,low,close,volume,tic,day
0,2010-01-04,33.580002,34.020000,33.450001,25.364527,7567500,XLB,0
1,2010-01-04,57.919998,58.810001,57.790001,37.747696,16928400,XLE,0
2,2010-01-04,11.795288,11.965881,11.770918,9.344016,92942347,XLF,0
3,2010-01-04,28.090000,28.320000,27.959999,21.793039,7471500,XLI,0
4,2010-01-04,23.139999,23.290001,23.100000,19.110195,8449400,XLK,0
...,...,...,...,...,...,...,...,...
30208,2023-05-04,147.660004,148.539993,146.929993,147.231598,5430500,XLK,3
30209,2023-05-04,76.739998,76.860001,76.220001,75.931320,11682600,XLP,3
30210,2023-05-04,68.190002,68.860001,67.550003,68.042702,14631100,XLU,3
30211,2023-05-04,132.899994,132.979996,131.839996,131.857208,9625300,XLV,3


In [8]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2010-01-04,33.580002,34.020000,33.450001,25.364527,7567500,XLB,0
1,2010-01-04,57.919998,58.810001,57.790001,37.747696,16928400,XLE,0
2,2010-01-04,11.795288,11.965881,11.770918,9.344016,92942347,XLF,0
3,2010-01-04,28.090000,28.320000,27.959999,21.793039,7471500,XLI,0
4,2010-01-04,23.139999,23.290001,23.100000,19.110195,8449400,XLK,0


### ETF 도메인 별로 최적의 하이퍼 파라미터 튜닝을 진행해보자

In [9]:
INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

INDICATORS는 FINRL 기본 베이스라인으로 해도 괜찮을 것 같음

https://github.com/jealous/stockstats


- change (in percent)
- delta
- permutation (zero-based)
- log return
- max in range
- min in range
- middle = (close + high + low) / 3
- compare: le, ge, lt, gt, eq, ne
- count: both backward(c) and forward(fc)
- cross: including upward cross and downward cross
- SMA: Simple Moving Average
- EMA: Exponential Moving Average
- MSTD: Moving Standard Deviation
- MVAR: Moving Variance
- RSV: Raw Stochastic Value
- RSI: Relative Strength Index
- KDJ: Stochastic Oscillator
- Bolling: Bollinger Band
- MACD: Moving Average Convergence Divergence
- CR: Energy Index (Intermediate Willingness Index)
- WR: Williams Overbought/Oversold index
- CCI: Commodity Channel Index
- TR: True Range
- ATR: Average True Range
- DMA: Different of Moving Average (10, 50)
- DMI: Directional Moving Index, including
- +DI: Positive Directional Indicator
- -DI: Negative Directional Indicator
- ADX: Average Directional Movement Index
- ADXR: Smoothed Moving Average of ADX
- TRIX: Triple Exponential Moving Average
- TEMA: Another Triple Exponential Moving Average
- VR: Volume Variation Index
- MFI: Money Flow Index
- VWMA: Volume Weighted Moving Average
- CHOP: Choppiness Index
- KAMA: Kaufman's Adaptive Moving Average
- PPO: Percentage Price Oscillator
- StochRSI: Stochastic RSI
- WT: LazyBear's Wave Trend
- Supertrend: with the Upper Band and Lower Band

In [10]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3356, 8)
Successfully added vix
Successfully added turbulence index


In [11]:
processed

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2010-01-04,33.580002,34.020000,33.450001,25.364527,7567500,XLB,0,0.000000,25.521516,25.289550,100.000000,66.666667,100.000000,25.364527,25.364527,20.040001,0.00000
1,2010-01-04,57.919998,58.810001,57.790001,37.747696,16928400,XLE,0,0.000000,25.521516,25.289550,100.000000,66.666667,100.000000,37.747696,37.747696,20.040001,0.00000
2,2010-01-04,11.795288,11.965881,11.770918,9.344016,92942347,XLF,0,0.000000,25.521516,25.289550,100.000000,66.666667,100.000000,9.344016,9.344016,20.040001,0.00000
3,2010-01-04,28.090000,28.320000,27.959999,21.793039,7471500,XLI,0,0.000000,25.521516,25.289550,100.000000,66.666667,100.000000,21.793039,21.793039,20.040001,0.00000
4,2010-01-04,23.139999,23.290001,23.100000,19.110195,8449400,XLK,0,0.000000,25.521516,25.289550,100.000000,66.666667,100.000000,19.110195,19.110195,20.040001,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30199,2023-05-03,149.690002,150.660004,148.179993,147.940125,5815100,XLK,2,1.101266,151.344400,144.908068,55.182961,46.442993,6.225843,147.699630,143.517544,18.340000,4.21999
30200,2023-05-03,77.059998,77.379997,76.639999,76.119987,11656600,XLP,2,0.875702,77.189108,74.079747,58.998060,93.547472,22.508083,74.795711,73.269687,18.340000,4.21999
30201,2023-05-03,68.639999,69.050003,67.989998,67.526855,11857200,XLU,2,0.319781,69.807814,67.590043,50.161291,8.126302,8.132944,67.692193,66.656867,18.340000,4.21999
30202,2023-05-03,134.259995,134.899994,133.270004,132.912888,8961500,XLV,2,0.915705,134.927106,131.520086,53.906948,54.096702,6.687990,131.318106,129.596837,18.340000,4.21999


In [12]:
list_ticker = processed["tic"].unique().tolist() #ticker 리스트 불러오기
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str)) #전체 데이터 날짜 날짜 리스트
combination = list(itertools.product(list_date,list_ticker)) #date, ticker 의 combination

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left") #date, tic combination 기준으로 우측에 해당되는 정보들 정리
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)
processed_full

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2010-01-04,XLB,33.580002,34.020000,33.450001,25.364527,7567500.0,0.0,0.000000,25.521516,25.289550,100.000000,66.666667,100.000000,25.364527,25.364527,20.040001,0.00000
1,2010-01-04,XLE,57.919998,58.810001,57.790001,37.747696,16928400.0,0.0,0.000000,25.521516,25.289550,100.000000,66.666667,100.000000,37.747696,37.747696,20.040001,0.00000
2,2010-01-04,XLF,11.795288,11.965881,11.770918,9.344016,92942347.0,0.0,0.000000,25.521516,25.289550,100.000000,66.666667,100.000000,9.344016,9.344016,20.040001,0.00000
3,2010-01-04,XLI,28.090000,28.320000,27.959999,21.793039,7471500.0,0.0,0.000000,25.521516,25.289550,100.000000,66.666667,100.000000,21.793039,21.793039,20.040001,0.00000
4,2010-01-04,XLK,23.139999,23.290001,23.100000,19.110195,8449400.0,0.0,0.000000,25.521516,25.289550,100.000000,66.666667,100.000000,19.110195,19.110195,20.040001,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43807,2023-05-03,XLK,149.690002,150.660004,148.179993,147.940125,5815100.0,2.0,1.101266,151.344400,144.908068,55.182961,46.442993,6.225843,147.699630,143.517544,18.340000,4.21999
43808,2023-05-03,XLP,77.059998,77.379997,76.639999,76.119987,11656600.0,2.0,0.875702,77.189108,74.079747,58.998060,93.547472,22.508083,74.795711,73.269687,18.340000,4.21999
43809,2023-05-03,XLU,68.639999,69.050003,67.989998,67.526855,11857200.0,2.0,0.319781,69.807814,67.590043,50.161291,8.126302,8.132944,67.692193,66.656867,18.340000,4.21999
43810,2023-05-03,XLV,134.259995,134.899994,133.270004,132.912888,8961500.0,2.0,0.915705,134.927106,131.520086,53.906948,54.096702,6.687990,131.318106,129.596837,18.340000,4.21999


### Data split

전체를 다 학습하면 안되는거 아닌가? 

In [13]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

26613
3591


### environment setting

In [14]:
stock_dimension = len(train.tic.unique()) 
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension #현재 상태를 나타내는 state를 정의. 1+ ticker 개수 x2 + 인디케이터 개수 x ticker개수 
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 9, State Space: 91


In [22]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


Earn_list =[]

for i in range(10):
  e_train_gym = StockTradingEnv(df = train, **env_kwargs)

  env_train, _ = e_train_gym.get_sb_env()



  if_using_a2c = True ##a2c만 사용해보자

  agent = DRLAgent(env = env_train)
  model_a2c = agent.get_model("a2c")

  if if_using_a2c:
    # set up logger
    tmp_path = RESULTS_DIR + '/a2c'
    new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
    # Set new logger
    model_a2c.set_logger(new_logger_a2c)
    
  trained_a2c = agent.train_model(model=model_a2c, 
                              tb_log_name='a2c',
                              total_timesteps=100000) if if_using_a2c else None

  data_risk_indicator = processed_full[(processed_full.date<TRAIN_END_DATE) & (processed_full.date>=TRAIN_START_DATE)]
  insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

  e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
  # env_trade, obs_trade = e_trade_gym.get_sb_env()

  trained_model = trained_a2c
  df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
      model=trained_model, 
      environment = e_trade_gym)

  df_result_a2c = df_account_value_a2c.set_index(df_account_value_a2c.columns[0])
  df_account_value_a2c.to_csv("df_account_value_a2c.csv")
  #baseline stats
  print("==============Get Baseline Stats===========")
  df_dji_ = get_baseline(
          ticker="^DJI", 
          start = TRADE_START_DATE,
          end = TRADE_END_DATE)
  stats = backtest_stats(df_dji_, value_col_name = 'close')
  df_dji = pd.DataFrame()
  df_dji['date'] = df_account_value_a2c['date']
  df_dji['account_value'] = df_dji_['close'] / df_dji_['close'][0] * env_kwargs["initial_amount"]
  df_dji.to_csv("df_dji.csv")
  df_dji = df_dji.set_index(df_dji.columns[0])
  df_dji.to_csv("df_dji+.csv")

  result = pd.DataFrame(df_result_a2c)


  result = pd.merge(result, df_dji, left_index=True, right_index=True)


  result.columns = ['a2c','dji']

  print("result: ", result)
  result.to_csv("result.csv")


  print('A2C로 얻은 투자 수익률>>', round((df_result_a2c.iloc[-1,0]/df_result_a2c.iloc[0,0]-1)*100,2),'%')
  Earn = round((df_result_a2c.iloc[-1,0]/df_result_a2c.iloc[0,0]-1)*100,2)
  
  Earn_list.append(Earn)



{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to results/a2c
---------------------------------------
| time/                 |             |
|    fps                | 179         |
|    iterations         | 100         |
|    time_elapsed       | 2           |
|    total_timesteps    | 500         |
| train/                |             |
|    entropy_loss       | -13         |
|    explained_variance | -0.198      |
|    learning_rate      | 0.0007      |
|    n_updates          | 99          |
|    policy_loss        | -22.2       |
|    reward             | 0.120480224 |
|    std                | 1.03        |
|    value_loss         | 3.8         |
---------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 178        |
|    iterations         | 200        |
|    time_elapsed       | 5          |
|    total_timesteps    | 1000       |
| train/                |   

c:\Users\se99a\anaconda3\lib\site-packages\pyfolio\timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


--------------------------------------
| time/                 |            |
|    fps                | 75         |
|    iterations         | 100        |
|    time_elapsed       | 6          |
|    total_timesteps    | 500        |
| train/                |            |
|    entropy_loss       | -12.9      |
|    explained_variance | -0.488     |
|    learning_rate      | 0.0007     |
|    n_updates          | 99         |
|    policy_loss        | -15.7      |
|    reward             | 0.38463306 |
|    std                | 1.01       |
|    value_loss         | 3.47       |
--------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 79        |
|    iterations         | 200       |
|    time_elapsed       | 12        |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -13       |
|    explained_variance | -0.02     |
|    learning_rate      | 0.0007  

c:\Users\se99a\anaconda3\lib\site-packages\pyfolio\timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


--------------------------------------
| time/                 |            |
|    fps                | 80         |
|    iterations         | 100        |
|    time_elapsed       | 6          |
|    total_timesteps    | 500        |
| train/                |            |
|    entropy_loss       | -12.9      |
|    explained_variance | -0.405     |
|    learning_rate      | 0.0007     |
|    n_updates          | 99         |
|    policy_loss        | -10.7      |
|    reward             | 0.03887803 |
|    std                | 1.02       |
|    value_loss         | 0.733      |
--------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 81         |
|    iterations         | 200        |
|    time_elapsed       | 12         |
|    total_timesteps    | 1000       |
| train/                |            |
|    entropy_loss       | -12.9      |
|    explained_variance | -0.176     |
|    learning_rate      |

...

c:\Users\se99a\anaconda3\lib\site-packages\pyfolio\timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


-----------------------------------------
| time/                 |               |
|    fps                | 74            |
|    iterations         | 100           |
|    time_elapsed       | 6             |
|    total_timesteps    | 500           |
| train/                |               |
|    entropy_loss       | -13.6         |
|    explained_variance | 5.96e-08      |
|    learning_rate      | 0.0007        |
|    n_updates          | 99            |
|    policy_loss        | -0.0239       |
|    reward             | -0.0004280027 |
|    std                | 1.1           |
|    value_loss         | 4.07e-06      |
-----------------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 76          |
|    iterations         | 200         |
|    time_elapsed       | 13          |
|    total_timesteps    | 1000        |
| train/                |             |
|    entropy_loss       | -14         |
|    exp

c:\Users\se99a\anaconda3\lib\site-packages\pyfolio\timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


--------------------------------------
| time/                 |            |
|    fps                | 75         |
|    iterations         | 100        |
|    time_elapsed       | 6          |
|    total_timesteps    | 500        |
| train/                |            |
|    entropy_loss       | -12.9      |
|    explained_variance | -0.148     |
|    learning_rate      | 0.0007     |
|    n_updates          | 99         |
|    policy_loss        | -17.3      |
|    reward             | 0.17835833 |
|    std                | 1.02       |
|    value_loss         | 2.45       |
--------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 77         |
|    iterations         | 200        |
|    time_elapsed       | 12         |
|    total_timesteps    | 1000       |
| train/                |            |
|    entropy_loss       | -12.9      |
|    explained_variance | 1.79e-07   |
|    learning_rate      |

c:\Users\se99a\anaconda3\lib\site-packages\pyfolio\timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


In [23]:
print('평균 수익률>>', round(np.mean(Earn_list),2),'%')
print('표준편차>>', round(np.std(Earn_list),2),'%')

평균 수익률>> -7.15 %
표준편차>> 6.95 %


In [24]:
pd.DataFrame(Earn_list, columns=['수익률'])

,수익률
0,-13.42
1,0.10
2,4.20
3,-8.26
4,-1.83
5,-13.84
6,-6.77
7,-11.94
8,-1.17
9,-18.53


In [29]:
pd.read_csv('Earn_list.csv').drop('Unnamed: 0',axis=1).describe()

,0
count,10.000000
mean,3.791000
std,17.341687
min,-18.600000
25%,-4.415000
50%,1.500000
75%,9.345000
max,35.820000
